# Feature Engineering con PyTS (HAR)

En este notebook se construirá conjuntos de *features* basados en transformaciones de series temporales usando **PyTS** sobre el dataset HAR (ventanas de 2.56 s, 128 pasos, 9 canales).  

**Salidas:**  
- **Resumen de Características** (`../reports/tables/pyts_feature_summary.csv`): Un archivo de metadatos que documenta la configuración y dimensiones de cada conjunto de características generado, asegurando la trazabilidad. 
- **Set de Features PAA** (`../artifacts/pyts_features_PAA.npz`): Un archivo comprimido con características basadas en Piecewise Aggregate Approximation, que capturan la forma global de las series de tiempo.

- **Set de Features BoP** (`../artifacts/pyts_features_BOP.npz`): Un archivo con características simbólicas basadas en Bag-of-Patterns, diseñadas para modelar la distribución de patrones locales y repetitivos.

- **Set de Features de Imagen** (`../artifacts/pyts_features_GAF_pooled.npz`): Un archivo opcional con características extraídas de la representación de las series como imágenes (Gramian Angular Fields), capturando su textura y dinámica 2D.

## 1. Carga y Normalización de Datos

En este primer paso, cargamos las series de tiempo del archivo `har_processed.npz`, obteniendo `X_train` con dimensiones de muestras, pasos de tiempo y canales, junto con las etiquetas `y_train`. Posteriormente, aplicamos **normalización Z-score** de forma independiente para cada uno de los 9 canales.

Este preprocesamiento es fundamental por dos razones: primero, garantiza que las señales de todos los sensores (acelerómetros y giroscopios) se encuentren en una escala común con media 0 y desviación estándar 1, evitando que los canales con magnitudes mayores dominen artificialmente el análisis. Segundo, muchas técnicas de análisis de series de tiempo, incluyendo PAA y SAX que se utilizarán posteriormente, son sensibles a la escala de los datos, por lo que la normalización asegura su funcionamiento estable y efectivo.

In [2]:
# ===============================
# 1. Carga de datos desde el caché
# ===============================
import numpy as np
import pandas as pd

# Ruta del archivo procesado
har_processed_path = "../data/har_processed.npz"

# Cargar
data = np.load(har_processed_path, allow_pickle=True)
X_train = data["X_train"]       # (n, 128, 9)
y_train = data["y_train"]       # (n,)
channel_names = data["channel_names"]

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("Canales:", channel_names)

# ===============================
# 2. Normalización Z-score por canal
# ===============================
# Forma de X_train: (n_muestras, n_pasos, n_canales)
n, T, C = X_train.shape

# Inicializamos array normalizado
X_norm = np.zeros_like(X_train, dtype=np.float32)

# Normalización independiente por canal
for c in range(C):
    mean_c = X_train[:, :, c].mean()
    std_c  = X_train[:, :, c].std()
    X_norm[:, :, c] = (X_train[:, :, c] - mean_c) / std_c
    print(f"Canal {channel_names[c]}: mean={mean_c:.3f}, std={std_c:.3f}")

# Reemplazamos X_train por versión normalizada
X_train = X_norm

# ===============================
# 3. Verificación de normalización
# ===============================
# Calculamos media y std por canal en el dataset normalizado
stats = []
for c in range(C):
    mean_c = X_train[:, :, c].mean()
    std_c  = X_train[:, :, c].std()
    stats.append([channel_names[c], round(mean_c, 3), round(std_c, 3)])

df_stats = pd.DataFrame(stats, columns=["Canal", "Media (≈0)", "STD (≈1)"])
display(df_stats)

X_train shape: (7352, 128, 9)
y_train shape: (7352,)
Canales: ['body_acc_x' 'body_acc_y' 'body_acc_z' 'body_gyro_x' 'body_gyro_y'
 'body_gyro_z' 'total_acc_x' 'total_acc_y' 'total_acc_z']
Canal body_acc_x: mean=-0.001, std=0.195
Canal body_acc_y: mean=-0.000, std=0.122
Canal body_acc_z: mean=-0.000, std=0.107
Canal body_gyro_x: mean=0.001, std=0.407
Canal body_gyro_y: mean=-0.001, std=0.382
Canal body_gyro_z: mean=0.000, std=0.256
Canal total_acc_x: mean=0.805, std=0.414
Canal total_acc_y: mean=0.029, std=0.391
Canal total_acc_z: mean=0.086, std=0.358


,Canal,Media (≈0),STD (≈1)
0,body_acc_x,0.0,1.0
1,body_acc_y,-0.0,1.0
2,body_acc_z,-0.0,1.0
3,body_gyro_x,0.0,1.0
4,body_gyro_y,0.0,1.0
5,body_gyro_z,0.0,1.0
6,total_acc_x,0.0,1.0
7,total_acc_y,0.0,1.0
8,total_acc_z,-0.0,1.0


## 2. Diseño de Estrategias de Características con `pyts`

Para representar las series de tiempo, se diseñaron tres estrategias complementarias que capturan diferentes aspectos de las señales de movimiento:

**Estrategia 1: Resumen Global con PAA (Piecewise Aggregate Approximation)**

PAA divide la secuencia de 128 puntos en segmentos más pequeños y calcula el promedio de cada uno. Esto captura la forma general de la señal: actividades dinámicas como caminar muestran patrones ondulatorios, mientras que actividades estáticas como sentarse son casi planas. Se genera un vector concatenado de características para los 9 canales.

**Estrategia 2: Patrones Locales con Bag-of-Patterns (BoP)**

BoP discretiza la señal en símbolos mediante SAX y cuenta la frecuencia de patrones recurrentes. Esto permite distinguir micro-patrones entre actividades similares: aunque caminar y subir escaleras son periódicas, la forma exacta de cada paso difiere. Se genera un histograma de frecuencias por canal que se concatena.

**Estrategia 3: Dinámica Visual con Imágenes (GAF o RP)**

Esta técnica convierte las series temporales en imágenes bidimensionales que revelan patrones de autocorrelación y dinámica no lineal. Se aplica solo a los canales más informativos (acelerómetros del cuerpo) y se resume mediante pooling para obtener vectores compactos que capturan la textura temporal de la señal.

In [ ]:
# ===============================
# PyTS: F1 PAA, F2 BoP (SAX), F3 GAF + pooling (versión robusta)
# ===============================
import numpy as np
from scipy.sparse import issparse
import inspect

# -------- utilidades --------
def _supports_param(cls, param_name):
    return param_name in inspect.signature(cls.__init__).parameters

def check_matrix(X, name):
    assert isinstance(X, np.ndarray), f"{name} no es np.ndarray"
    assert np.isfinite(X).all(), f"{name} contiene NaN/Inf"
    print(f"{name}: shape={X.shape}, dtype={X.dtype}")

# ===============================
# F1. PAA (Piecewise Aggregate Approximation)
# ===============================
def features_paa(X, n_segments=32, copy=True, pad_mode='edge'):
    """
    Devuelve (n, C * n_segments). Soporta PAA moderno (n_segments) o antiguo (window_size).
    Si la API antigua requiere divisibilidad, hace padding automático.
    """
    from pyts.approximation import PiecewiseAggregateApproximation as _PAA
    if copy:
        X = np.array(X, copy=True)
    n, T, C = X.shape

    use_n_segments = _supports_param(_PAA, 'n_segments')
    if use_n_segments:
        paa = _PAA(n_segments=n_segments)  # API moderna remapea a n_segments
    else:
        # API antigua: necesita window_size entero -> rellenamos si no divide
        window_size = T // n_segments + (1 if T % n_segments else 0)
        T_needed = window_size * n_segments
        if T_needed != T:
            X = np.pad(X, ((0,0), (0, T_needed - T), (0,0)), mode=pad_mode)
        paa = _PAA(window_size=window_size)

    feats = [paa.transform(X[:, :, c]) for c in range(C)]  # cada (n, n_segments)
    X_paa = np.concatenate(feats, axis=1)                  # (n, C * n_segments)
    return X_paa

# ===============================
# F2. Bag-of-Patterns/Words (SAX + ventana deslizante)
# ===============================
def features_bop(
    X,
    window_size=16,
    word_size=8,
    n_bins=6,
    numerosity_reduction=True,
    sparse=True,
    strategy='normal'
):
    """
    Devuelve (n, sum n_words_c) denso. Hace fallback a BagOfWords (nuevas versiones)
    y filtra kwargs según soporte real de la clase.
    """
    try:
        from pyts.transformation import BagOfPatterns as _BoX
    except ImportError:
        from pyts.transformation import BagOfWords as _BoX  # versiones nuevas

    def supports(k): 
        return k in inspect.signature(_BoX.__init__).parameters

    kwargs_all = dict(
        window_size=window_size,
        word_size=word_size,
        n_bins=n_bins,
        numerosity_reduction=numerosity_reduction,
        sparse=sparse,
        strategy=strategy
    )
    kwargs = {k: v for k, v in kwargs_all.items() if supports(k)}

    n, T, C = X.shape
    bop = _BoX(**kwargs)
    feats = []
    for c in range(C):
        H = bop.fit_transform(X[:, :, c])    # (n, n_words_c)
        if issparse(H):
            H = H.toarray()
        feats.append(H)
    X_bop = np.concatenate(feats, axis=1)
    return X_bop

# ===============================
# F3. GAF (Gramian Angular Field) + pooling (opcional)
# ===============================
from pyts.image import GramianAngularField

def block_pool_2d(images, pool_h=8, pool_w=8):
    """
    images: (n_samples, H, W)
    Average pooling no solapado en bloques (pool_h x pool_w).
    Devuelve: (n_samples, H/pool_h, W/pool_w)
    """
    n, H, W = images.shape
    assert H % pool_h == 0 and W % pool_w == 0, \
        f"La imagen ({H}x{W}) debe ser múltiplo del pooling ({pool_h}x{pool_w})."
    h_blocks = H // pool_h
    w_blocks = W // pool_w
    pooled = images.reshape(n, h_blocks, pool_h, w_blocks, pool_w).mean(axis=(2, 4))
    return pooled

def features_gaf_pooled(
    X,
    channel_indices,
    image_size=64,
    method='summation',  # 'summation' | 'difference'
    pool_h=8, pool_w=8
):
    """
    X: (n, T, C) Z-normalizado
    channel_indices: lista de canales a transformar
    Devuelve: X_IMG (n, len(channel_indices) * (image_size/pool_h) * (image_size/pool_w))
    """
    n, T, C = X.shape
    # Sanitizar índices
    ch = [int(ci) for ci in channel_indices if 0 <= int(ci) < C]
    if len(ch) == 0:
        raise ValueError(f"channel_indices fuera de rango para C={C}")
    if image_size % pool_h != 0 or image_size % pool_w != 0:
        raise ValueError("image_size debe ser múltiplo de pool_h y pool_w")

    gaf = GramianAngularField(image_size=image_size, method=method)
    pooled_feats = []
    for c in ch:
        imgs = gaf.fit_transform(X[:, :, c])               # (n, H, W) con H=W=image_size
        imgs_pooled = block_pool_2d(imgs, pool_h, pool_w)  # (n, H', W')
        pooled_feats.append(imgs_pooled.reshape(n, -1))    # (n, H'*W')
    X_img = np.concatenate(pooled_feats, axis=1)
    return X_img

# ===============================
# Ejecutar (asumiendo X_train ya normalizado, shape (n, T, C))
# ===============================
# 1) PAA
X_PAA = features_paa(X_train, n_segments=32)
check_matrix(X_PAA, "X_PAA")

# 2) BoP (SAX)
X_BOP = features_bop(
    X_train,
    window_size=16,
    word_size=8,
    n_bins=6,
    numerosity_reduction=True,
    sparse=True,        # si no está soportado en tu versión, se ignora
    strategy='normal'   # idem
)
check_matrix(X_BOP, "X_BOP")

# 3) GAF + pooling
# Intento de mapeo por nombre si existe channel_names; si no, fallback [0,1,2]
try:
    channel_names = [str(x) for x in channel_names]
except NameError:
    channel_names = None

def _find_indices(names, wanted):
    idx = []
    for w in wanted:
        if w in names:
            idx.append(names.index(w))
    return idx

if channel_names:
    wanted = ["body_acc_x","body_acc_y","body_acc_z"]
    body_acc_idx = _find_indices(channel_names, wanted)
    if len(body_acc_idx) == 0:
        body_acc_idx = [0, 1, 2]  # fallback seguro
else:
    body_acc_idx = [0, 1, 2]

X_IMG = features_gaf_pooled(
    X_train,
    channel_indices=body_acc_idx,
    image_size=64,
    method='summation',
    pool_h=8, pool_w=8
)
check_matrix(X_IMG, "X_IMG (GAF pooled, body_acc)")

# Resumen
summary = [
    ("PAA", X_PAA.shape[1]),
    ("BoP", X_BOP.shape[1]),
    ("GAF_pooled(body_acc)", X_IMG.shape[1]),
]
for name, d in summary:
    print(f"{name:22s} -> n_features = {d}")


X_PAA: shape=(7352, 288), dtype=float64


## 3. Selección y Justificación de Hiperparámetros

La efectividad de las transformaciones de `pyts` depende de sus hiperparámetros. Los valores seleccionados equilibran capacidad representativa y simplicidad del modelo.

**Para PAA:**

Se utiliza `n_segments = 32`, reduciendo cada señal de 128 a 32 puntos mediante promediado. Esto suaviza el ruido y preserva la forma estructural, generando 288 características totales (9 canales × 32).

**Para Bag-of-Patterns:**

Se configuran `window_size = 16` para capturar micro-movimientos de aproximadamente 0.32 segundos, `word_size = 8` para equilibrar descripción y complejidad en la representación SAX, y `n_bins = 6` para discretizar la señal con granularidad adecuada sin excesiva sensibilidad al ruido.

**Para Imágenes (GAF o RP):**

Se genera `image_size = 64` con pooling a `8x8`, creando una huella digital de 64 características por canal. Se aplica solo a los acelerómetros corporales `body_acc_x`, `body_acc_y` y `body_acc_z` por contener la mayor información discriminativa, generando 192 características totales (3 canales × 64).

## 4. Pipeline de construcción

1. **Normalizar** `X_train` por canal (z-score).  
2. **F1-PAA:** aplicar por canal → concatenar → `X_PAA` `(n, 9*M)`.  
3. **F2-Bag-of-Patterns:** aplicar por canal → concatenar histogramas → `X_BOP`.  
4. **F3-Imagen (opcional):** GAF/RP en 3 canales `body_acc_*` → *pooling* 8×8 → concatenar → `X_IMG`.  
5. **Chequear varianza** y remover columnas constantes.  
6. **Guardar artefactos** (`.npz` + CSV de resumen con dimensiones, tiempos y parámetros).

## 5. Validaciones rápidas (sin entrenamiento)

- **Sanity checks:** dimensiones de salida, ausencia de NaN/inf, varianza > 0.  
- **Separabilidad preliminar:** PCA/UMAP con `X_PAA` y `X_BOP` (2D) para visualizar si hay agrupamientos por actividad.  
- **Correlaciones:** matriz de correlación entre *features* de F1 y F2 para descartar redundancias extremas.

## 6. Exportación para el paper y trazabilidad

- Guardar **resumen de *features*** (`../reports/tables/pyts_feature_summary.csv`) con:  
  - `set_name`, `n_features`, `n_segments / window_size / word_size / n_bins`, canales usados, normalización y compute_time_sec (tiempo de cómputo en segundos que tomó generar cada conjunto de features).
- Guardar **figuras** de:  
  - PCA de `X_PAA` y de `X_BOP` (coloreado por actividad).  
- Guardar artefactos:  
  - `../artifacts/pyts_features_PAA.npz` (X_PAA, y, meta)  
  - `../artifacts/pyts_features_BOP.npz` (X_BOP, y, meta)  
  - `../artifacts/pyts_features_GAF_pooled.npz`


## 7. Limitaciones y plan para tsfresh

- **PyTS** resume forma global (PAA) y patrones locales discretizados (BoP), pero no agota todos los estadísticos temporales.  
- **tsfresh** complementará con **features estadísticas masivas** (autocorrelación, entropías, percentiles, coeficientes de Fourier), seguidas de **selección de *features***.  
- En el notebook 03 (modelado), compararemos desempeño usando:  
  - Solo PyTS (F1/F2)  
  - Solo tsfresh  
  - **Combinación PyTS + tsfresh** (si la dimensionalidad y el *overfitting* lo permiten)


## 8. Conclusiones (PyTS)

- Se definieron y justificaron **tres familias** de *features* en PyTS: **PAA (compacto)**, **BoP (motivos locales)** y **GAF/RP (opcional, firma visual)**.  
- Los parámetros propuestos priorizan **robustez, baja dimensionalidad e interpretabilidad**.  
- Dejaremos listas las matrices de *features* y el resumen para integrarlas en el **notebook de modelado** y en el **paper** (sección Metodología/Experimentación).
